
#This task can be accomplished in a few ways, as listed below.



Using multi-modal models such as LiLT,Donut, LayoutLM, etc., this method needs training data to train these models and has to apply post-processing steps to bring out the expected format.



1. Using AWS textract to extract the tables from the document and applying post-processing techniques to bring out the expected format.


2. using LLM's to get the information out of the documents. Here, we can use two types of llms.

  1. closed-source models or paid services like OpenAI, Claude, and other models.
  2. open source models like Mistral, Zephyr, and others.



In this note book, I have used only llms to extract the information. Here, I have demonstrated in different ways how we can extract the information from the document. first one is using prompt and second one is using schema, schema contain predifined keys need to be extracted from text.But using prompt we can all the information in key value pairs but lot of post processing and prompt optimizing is required.

***Note:*** In this demo, I haven't used any OCR or document loading techniques. This can be done using paddle ocr, AWS textract, gocr, and tesseract. We can easily extract the text from the document. So, I haven't used any approaches here.


# Using openai

In [ ]:
!pip install langchain_openai

In [46]:
import os
os.environ["OPENAI_API_KEY"] = "openai key"

In [53]:
import os
from langchain_openai import OpenAI

llm = OpenAI(max_tokens= 2000)

##using schemas

In [49]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)

In [50]:
from langchain.chains import create_extraction_chain,create_extraction_chain_pydantic

In [51]:
input_text = """Price List  DEMO-CATALOG WWW: http://www.company.com E-mail: mail@company.com Phone: 206-xxx-0822 P.O. Box 34069 #381 Seattle, WA 981xx-10xx USA New! Bestseller Sale Action! Not available
                Code Description Price Women's Bags WB001 AMELIEGALANTI 2016 new casual women shoulder bags famous brand Colors:White,Beige[+5],Blue;Set:Shoes[+100] 12,00
                WB002 SheIn Bags Handbags Women Famous Brands Ladies Fashion 16,00 WB003 2016 New fashion shoulder bags handbags women famous brand 17,00 WB004 Fashion Small Bag Women Messenger Bags Soft PU Leather Handbags 22,00
                WB005 Famous Brand Luxury Women Leather Handbags Women's Trunk 55,66 WB006 Realer brand women bag hollow out ombre handbag floral 22,00 WB007 New Arrival Women Fashion Faux Leather Satchel Bowknot 12,00
                WB008 2016 New Fashion Women PU Leather Handbag High Quality 34,00 WB009 Ladsoul 2016 women handbag pu leather women shoulder 51,00 WB0010 Famous Brand Ladies Hand Bags Women PU Leather Bag Brown 54,00
                WB0011 2016 New fashion women bag vintage handbags crossbody 67,00 """

In [60]:
schema =  {
          "type": "object",
          "properties": {"SKU ID": {"type": "string"},
                          "product description": {"type": "string"},
                          "item title": {"type": "string"},
                          "price": {"type": "string"},
                          "attributes": {"type": "string"}},
          "required": ["SKU ID","product description","item name","price","attributes"],
        }

In [61]:
chain = create_extraction_chain(schema,llm)

In [62]:
chain.run(input_text)

[{'SKU ID': 'WB001',
  'product description': 'AMELIEGALANTI 2016 new casual women shoulder bags famous brand',
  'price': '12,00',
  'attributes': 'Colors:White,Beige[+5],Blue;Set:Shoes[+100]'},
 {'SKU ID': 'WB002',
  'product description': 'SheIn Bags Handbags Women Famous Brands Ladies Fashion',
  'price': '16,00',
  'attributes': ''},
 {'SKU ID': 'WB003',
  'product description': '2016 New fashion shoulder bags handbags women famous brand',
  'price': '17,00',
  'attributes': ''},
 {'SKU ID': 'WB004',
  'product description': 'Fashion Small Bag Women Messenger Bags Soft PU Leather Handbags',
  'price': '22,00',
  'attributes': ''},
 {'SKU ID': 'WB005',
  'product description': "Famous Brand Luxury Women Leather Handbags Women's Trunk",
  'price': '55,66',
  'attributes': ''},
 {'SKU ID': 'WB006',
  'product description': 'Realer brand women bag hollow out ombre handbag floral',
  'price': '22,00',
  'attributes': ''},
 {'SKU ID': 'WB007',
  'product description': 'New Arrival Women

##Using langchain chains Example :1

In [44]:
from langchain.prompts import PromptTemplate

template = '''You are the best ocr extractor, document summariser, information extraction expert in the world with e-commerce domain knowledge related to e-commerce documents, like catalogue documents, etc.
              from the {input_text} extract all the key value pairs present. the key-value pair can be in different formats as below
              1. Key can be a header and value can be the text below it.
              2. Key is left and value at its right with colon seperation
              3. Key is the column header of a table and value below it
              for all those cases give the key and corresponding value text.
              give the output as a json structure.

              Import thing to strictly follow.
              1. Ensure that if there are variations of the same SKU ID, each variation is treated as a separate SKU.
              2. The expected output format for each extracted SKU should be a structured object like this: "ID": 848063, "title": "", "description": "Loctite ...", "price":972, "attributes": "Pack Size": "50 ml".
              3. Ensure that all structured product information includes an ID, title, description, price, and
                attributes. Any additional information should be stored as key-value pairs within the
                attributes section.
                          '''
prompt = PromptTemplate.from_template(template)



input_text = """Price List  DEMO-CATALOG WWW: http://www.company.com E-mail: mail@company.com Phone: 206-xxx-0822 P.O. Box 34069 #381 Seattle, WA 981xx-10xx USA New! Bestseller Sale Action! Not available
                Code Description Price Women's Bags WB001 AMELIEGALANTI 2016 new casual women shoulder bags famous brand Colors:White,Beige[+5],Blue;Set:Shoes[+100] 12,00
                WB002 SheIn Bags Handbags Women Famous Brands Ladies Fashion 16,00 WB003 2016 New fashion shoulder bags handbags women famous brand 17,00 WB004 Fashion Small Bag Women Messenger Bags Soft PU Leather Handbags 22,00
                WB005 Famous Brand Luxury Women Leather Handbags Women's Trunk 55,66 WB006 Realer brand women bag hollow out ombre handbag floral 22,00 WB007 New Arrival Women Fashion Faux Leather Satchel Bowknot 12,00
                WB008 2016 New Fashion Women PU Leather Handbag High Quality 34,00 WB009 Ladsoul 2016 women handbag pu leather women shoulder 51,00 WB0010 Famous Brand Ladies Hand Bags Women PU Leather Bag Brown 54,00
                WB0011 2016 New fashion women bag vintage handbags crossbody 67,00 """



In [54]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [55]:
llm_chain.run(input_text)

'\n{\n  "SKU001": {\n    "ID": "WB001",\n    "title": "AMELIEGALANTI 2016 new casual women shoulder bags famous brand",\n    "description": "Colors:White,Beige[+5],Blue;Set:Shoes[+100]",\n    "price": "12,00",\n    "attributes": {\n      "Colors": "White,Beige[+5],Blue",\n      "Set": "Shoes[+100]"\n    }\n  },\n  "SKU002": {\n    "ID": "WB002",\n    "title": "SheIn Bags Handbags Women Famous Brands Ladies Fashion",\n    "description": "",\n    "price": "16,00",\n    "attributes": {}\n  },\n  "SKU003": {\n    "ID": "WB003",\n    "title": "2016 New fashion shoulder bags handbags women famous brand",\n    "description": "",\n    "price": "17,00",\n    "attributes": {}\n  },\n  "SKU004": {\n    "ID": "WB004",\n    "title": "Fashion Small Bag Women Messenger Bags Soft PU Leather Handbags",\n    "description": "",\n    "price": "22,00",\n    "attributes": {}\n  },\n  "SKU005": {\n    "ID": "WB005",\n    "title": "Famous Brand Luxury Women Leather Handbags Women\'s Trunk",\n    "description"

##Example:2

In [56]:
from langchain.prompts import PromptTemplate

template = '''You are the best ocr extractor, document summariser, information extraction expert in the world with e-commerce domain knowledge related to e-commerce documents, like catalogue documents, etc.
              from the {input_text} extract all the key value pairs present. the key-value pair can be in different formats as below
              1. Key can be a header and value can be the text below it.
              2. Key is left and value at its right with colon seperation
              3. Key is the column header of a table and value below it
              for all those cases give the key and corresponding value text.
              give the output as a json structure.

              Import thing to strictly follow.
              1. Ensure that if there are variations of the same SKU ID, each variation is treated as a separate SKU.
              2. The expected output format for each extracted SKU should be a structured object like this: "ID": 848063, "title": "", "description": "Loctite ...", "price":972, "attributes": "Pack Size": "50 ml".
              3. Ensure that all structured product information includes an ID, title, description, price, and
                attributes. Any additional information should be stored as key-value pairs within the
                attributes section.
                          '''
prompt = PromptTemplate.from_template(template)



input_text = """Code Description Price
Women's Bags
WB001 AMELIEGALANTI 2016 new casual
women shoulder bags famous brand 12,00
WB002 SheIn Bags Handbags Women Famous
Brands Ladies Fashion 16,00
WB003 2016 New fashion shoulder bags
handbags women famous brand 17,00
WB004 Fashion Small Bag Women Messenger
Bags Soft PU Leather Handbags 22,00
WB005 Famous Brand Luxury Women Leather
Handbags Women's Trunk 55,66
WB006 Realer brand women bag hollow out
ombre handbag floral 22,00
WB007 New Arrival Women Fashion Faux
Leather Satchel Bowknot 12,00
WB008 2016 New Fashion Women PU Leather
Handbag High Quality 34,00
WB009 Ladsoul 2016 women handbag pu
leather women shoulder 51,00
WB0010 Famous Brand Ladies Hand Bags
Women PU Leather Bag Brown 54,00
WB0011 2016 New fashion women bag vintage
handbags crossbody 67,00
WB0012 New Brand Bags For Women Leather
Women Messenger 54,00
WB0013 INLEELA 2016 Fashion Scrub Women
Bucket Bag Vintage 32,00
WB0014 Women Shoulder Bag Candy 10 Colors
Fashion Brand 44,00
WB0015 Hot selling! Women Fashion Small
Shell Leather Plaid 56,00
WB0016 Fashion 2016 Designers Women
Messenger Bags Females 78,00
Women's Bags - More
WB0017 Mance Summer Canvas Women Beach
Bag Fashion Color 98,00
WB0018 2016 new woman handbag famous
brands shoulder bag Ladies 66,00
WB0019 FLYING BIRDS! women leather
handbag messenger bags shoulder 54,00
WB0020 Bolsa Bolsos Carteras Mujer Marca
Women PU Leather Cat 34,00
WB0021 2016 NEW Fashion Famous Brand PU
Leather Woven Shoulder 52,00
WB0023 New Fashion Alligator Lady Embossing
Messenger Crocodile 23,00
WB0022 All-Match Fashion Female Package
2015 Winter Elegant Women 61,00 """



In [57]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [58]:
llm_chain.run(input_text)

'\n{\n  "WB001": {\n    "ID": "WB001",\n    "title": "AMELIEGALANTI 2016 new casual women shoulder bags famous brand",\n    "description": "This is a 2016 new casual women shoulder bag from the famous brand AMELIEGALANTI. It is perfect for everyday use and adds a touch of luxury to any outfit.",\n    "price": 12.00,\n    "attributes": {\n      "Brand": "AMELIEGALANTI",\n      "Year": "2016",\n      "Style": "Casual",\n      "Type": "Shoulder Bag",\n      "Material": "Unknown"\n    }\n  },\n  "WB002": {\n    "ID": "WB002",\n    "title": "SheIn Bags Handbags Women Famous Brands Ladies Fashion",\n    "description": "This is a fashionable handbag from the brand SheIn. It features a trendy design and is perfect for any occasion.",\n    "price": 16.00,\n    "attributes": {\n      "Brand": "SheIn",\n      "Type": "Handbag",\n      "Material": "Unknown",\n      "Style": "Fashion",\n      "Gender": "Women"\n    }\n  },\n  "WB003": {\n    "ID": "WB003",\n    "title": "2016 New fashion shoulder b

#using mistral and langchain [Local LLM]

In [ ]:
!pip install langchain

In [ ]:
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7

In [ ]:
!pip install -U transformers

In [2]:
import transformers
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your
fantasy football knowledge. Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
llm_chain.invoke({"context":"",
                  "question": "Should I pick up Alvin Kamara for my fantasy team?"})


#using langchain and llamacpp  [Local LLM]

In [ ]:
!pip install -U langchain-community langchain
!pip install -U transformers
! pip install llama-cpp-python


In [7]:
#Downloading model from huggingface

!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
# https://huggingface.co/TheBloke/phi-2-GGUF/blob/main/phi-2.Q6_K.gguf
#https://huggingface.co/mlabonne/gemma-2b-GGUF/blob/main/gemma-2b.Q8_0.gguf
#https://huggingface.co/TheBloke/MPT-7B-Instruct-GGML/blob/main/mpt-7b-instruct.ggmlv3.q4_1.bin
#https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/blob/main/mistral-7b-v0.1.Q4_K_M.gguf

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpgsedq7wk
mistral-7b-instruct-v0.1.Q4_K_M.gguf: 100% 4.37G/4.37G [00:19<00:00, 224MB/s]
./mistral-7b-instruct-v0.1.Q4_K_M.gguf


In [18]:
#preparing prompt and templeted
from langchain.prompts import PromptTemplate

template = '''You are the best ocr extractor, document summariser, information extraction expert in the world with e-commerce domain knowledge related to e-commerce documents, like catalogue documents, etc.
              from this {input_text} context ,This context may contain multiple sku details, and it extracted from ocr, do analyse the values and extract all the key value pairs present. the key-value pair can be in different formats as below
              1. Key can be a header and value can be the text below it.
              2. Key is left and value at its right with colon seperation
              3. Key is the column header of a table and value below it
              for all those cases give the key and corresponding value text.
              give the output as a json structure.


                          '''
prompt = PromptTemplate.from_template(template)

# chain = prompt | hf

input_text = """Price List  DEMO-CATALOG WWW: http://www.company.com E-mail: mail@company.com Phone: 206-xxx-0822 P.O. Box 34069 #381 Seattle, WA 981xx-10xx USA New! Bestseller Sale Action! Not available
                Code Description Price Women's Bags WB001 AMELIEGALANTI 2016 new casual women shoulder bags famous brand Colors:White,Beige[+5],Blue;Set:Shoes[+100] 12,00
                WB002 SheIn Bags Handbags Women Famous Brands Ladies Fashion 16,00 WB003 2016 New fashion shoulder bags handbags women famous brand 17,00 WB004 Fashion Small Bag Women Messenger Bags Soft PU Leather Handbags 22,00
                WB005 Famous Brand Luxury Women Leather Handbags Women's Trunk 55,66 WB006 Realer brand women bag hollow out ombre handbag floral 22,00 WB007 New Arrival Women Fashion Faux Leather Satchel Bowknot 12,00
                WB008 2016 New Fashion Women PU Leather Handbag High Quality 34,00 WB009 Ladsoul 2016 women handbag pu leather women shoulder 51,00 WB0010 Famous Brand Ladies Hand Bags Women PU Leather Bag Brown 54,00
                WB0011 2016 New fashion women bag vintage handbags crossbody 67,00 """

# print(chain.invoke({"input_text": input_text}))

##using llamacpp and chains from langchain

In [19]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import os
#loading the model

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

n_gpu_layers = 35  # Change this value based on your model and your GPU VRAM pool.
n_batch = 64  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.1,
    n_ctx=1000,
    max_tokens=2000,
    n_threads = os.cpu_count(),
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llm_chain = LLMChain(prompt=prompt, llm=llm)



llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

In [20]:
llm_chain.run({"input_text":input_text})

 {
                               "SKU": "WB001",
                               "Brand": "AMELIEGALANTI",
                               "Description": "2016 new casual women shoulder bags famous brand Colors:White,Beige[+5],Blue;Set:Shoes[+100]",
                               "Price": "12,00",
                               "Category": "Women's Bags",
                               "Subcategory": "Shoulder Bags",
                               "Color": "White",
                               "Set": "Shoes",
                               "Quantity": "1",
                               "Sale": "Best Seller Sale",
                               "Action": "Not available"
                           }
                           {
                               "SKU": "WB002",
                               "Brand": "SheIn",
                               "Description": "Bags Handbags Women Famous Brands Ladies Fashion 16,00",
                               "Price": "16,00",
             


llama_print_timings:        load time =    8123.89 ms
llama_print_timings:      sample time =     257.83 ms /   403 runs   (    0.64 ms per token,  1563.02 tokens per second)
llama_print_timings: prompt eval time =   81933.99 ms /   597 tokens (  137.24 ms per token,     7.29 tokens per second)
llama_print_timings:        eval time =   75648.08 ms /   402 runs   (  188.18 ms per token,     5.31 tokens per second)
llama_print_timings:       total time =  159891.01 ms /   999 tokens


' {\n                               "SKU": "WB001",\n                               "Brand": "AMELIEGALANTI",\n                               "Description": "2016 new casual women shoulder bags famous brand Colors:White,Beige[+5],Blue;Set:Shoes[+100]",\n                               "Price": "12,00",\n                               "Category": "Women\'s Bags",\n                               "Subcategory": "Shoulder Bags",\n                               "Color": "White",\n                               "Set": "Shoes",\n                               "Quantity": "1",\n                               "Sale": "Best Seller Sale",\n                               "Action": "Not available"\n                           }\n                           {\n                               "SKU": "WB002",\n                               "Brand": "SheIn",\n                               "Description": "Bags Handbags Women Famous Brands Ladies Fashion 16,00",\n                               "Price": "16

##using llama_cpp library and schema





In [ ]:
from llama_cpp import Llama
llm = Llama(
      model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
      n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      n_ctx=1000,
    max_tokens=1000, # Uncomment to increase the context window
      chat_format="chatml"
)


In [37]:
output = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": """You are a helpful assistant that outputs in JSON.""",
        },
        {"role": "user", "content": input_text},
    ],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {"SKU ID": {"type": "string"},
                           "product description": {"type": "string"},
                           "item title": {"type": "string"},
                           "price": {"type": "float"},
                           "attributes": {"type": "string"}},
            "required": ["id","product description","item name","price","attributes"],
        },
    },
    temperature=0.3,
)
output

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

{'id': 'chatcmpl-689211a6-abd0-4a6f-97e2-d2d339f59ca0',
 'object': 'chat.completion',
 'created': 1708880923,
 'model': 'mistral-7b-instruct-v0.1.Q4_K_M.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{\n   "PRICE LIST":{\n      "DEMO-CATALOG":{\n         "WWW": "http://www.company.com",\n         "EMAIL": "mail@company.com",\n         "PHONE": "206-xxx-0822",\n         "PO BOX": "P.O. Box 34069 #381 Seattle, WA 981xx-10xx USA",\n         "NEW": "Best Seller Sale Action!",\n         "CODES": [\n            {\n               "DESCRIPTION": "Women\'s Bags",\n               "PRICE": "12,00",\n               "SET": "Shoes[+100]"\n            },\n            {\n               "DESCRIPTION": "SheIn Bags Handbags Women Famous Brands Ladies Fashion",\n               "PRICE": "16,00"\n            },\n            {\n               "DESCRIPTION": "2016 New fashion shoulder bags handbags women famous brand",\n               "PRICE": "17,00"\n            },\n   

In [38]:
output['choices'][0]['message']['content']

'{\n   "PRICE LIST":{\n      "DEMO-CATALOG":{\n         "WWW": "http://www.company.com",\n         "EMAIL": "mail@company.com",\n         "PHONE": "206-xxx-0822",\n         "PO BOX": "P.O. Box 34069 #381 Seattle, WA 981xx-10xx USA",\n         "NEW": "Best Seller Sale Action!",\n         "CODES": [\n            {\n               "DESCRIPTION": "Women\'s Bags",\n               "PRICE": "12,00",\n               "SET": "Shoes[+100]"\n            },\n            {\n               "DESCRIPTION": "SheIn Bags Handbags Women Famous Brands Ladies Fashion",\n               "PRICE": "16,00"\n            },\n            {\n               "DESCRIPTION": "2016 New fashion shoulder bags handbags women famous brand",\n               "PRICE": "17,00"\n            },\n            {\n               "DESCRIPTION": "Fashion Small Bag Women Messenger Bags Soft PU Leather Handbags",\n               "PRICE": "22,00"\n            },\n            {\n               "DESCRIPTION": "Famous Brand Luxury Women Leathe